## 1. Stohastic block model

### (a) Consider a network with adjacency matrix: $$\begin{bmatrix} 0 & 1 & 1 & 0 & 0 & 0 & 0 \\ 1 & 0 & 1 & 0 & 0 & 0 & 0 \\ 1 & 1 & 0 & 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 & 1 & 1 & 0 \\ 0 & 0 & 0 & 1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 & 1 & 0 & 1 \\ 0 & 0 & 0 & 0 & 0 & 1 & 0 \end{bmatrix}$$ and a stohastic block model with block assignment vector $\hat{z}$ = (0, 0, 0, 1, 1, 1, 1). Give a maximum likelihood estimation for the stohastic block matrix M. Is this the vector $\vec{z}$ that maximizes the likelihood function for B = 2?

First of all, let`s create the networks. 

In [1]:
from decimal import *
from collections import Counter


import numpy as np
import scipy as sp
import pathpy as pp
import seaborn as sns
from tqdm import tqdm


sns.set_style("whitegrid")

In [2]:
network = pp.Network(directed=False)
network.add_edge("0", "1")
network.add_edge("0", "2")
network.add_edge("1", "2")
network.add_edge("2", "3")
network.add_edge("3", "4")
network.add_edge("3", "5")
network.add_edge("4", "5")
network.add_edge("5", "6")
network.plot(edge_color="grey")

In [3]:
def color_graph_communities(network: pp.Network, z_detected):
    number_of_blocks = len(set(z_detected))
    colors = sns.color_palette("Set1", number_of_blocks)
    for node in network.nodes:
        node['color'] = colors[z_detected[network.nodes.index[node.uid]]]


z = (0, 0, 0, 1, 1, 1, 1)
color_graph_communities(network, z)
network.plot(edge_color="grey")

The formula for calculating maximum likelihood estimation for the block matrix entry is:
$$\hat{M_{kl}} = N_{lk} / E_{lk}$$
,where: 

- $N_{lk}$ - the number of possible links between  clusters l and k;
- $E_{lk}$ - the number of existing links between nodes in clusters l and k.


In [4]:
def estimate_block_matrix(network, z):
    
    # B is the number of blocks
    B = len(set(z))

    # C[k] counts the number of nodes in block k
    C = Counter(z)

    # E[k,l] counts how many links exist between nodes in block k and block l
    E = np.zeros((B,B))
    for e in network.edges:
        if e.v.uid != e.w.uid:     
            E[z[network.nodes.index[e.v.uid]],z[network.nodes.index[e.w.uid]]] += 1
            # increment count for both directions
            if z[network.nodes.index[e.v.uid]] != z[network.nodes.index[e.w.uid]]:
                E[z[network.nodes.index[e.w.uid]],z[network.nodes.index[e.v.uid]]] += 1

    # N[k,l] counts how many links can possibly exist between nodes in block k and l
    N = np.zeros((B,B))

    # calculate number of possible links N[k,l] for all pairs of blocks k, l (see comments above)
    for k in range(B):
        for l in range(B):            
            if k == l:
                N[k,l] = sp.special.binom(C[k], 2)
            else:
                N[k,l]= C[k] * C[l]

    # estimate block matrix entries by dividing number of observed edges between k, l by number of edges possible between k, l
    M = np.zeros((B,B))
    for k in range(B):
        for l in range(B):
            M[k,l] = E[k,l] / N[k,l]
    return M

In [5]:
M = estimate_block_matrix(network, z)
M

array([[1.        , 0.08333333],
       [0.08333333, 0.66666667]])

Let`s use the simuled annualing algorithm for defining the maximum likelihood function for two blocks.

In [6]:
def log(x):
    if x == 0:
        return 0.0
    else:
        return np.log(x)


def SBM_likelihood(z, n: pp.Network):
    B = len(set(z))

    # C[k] counts number of nodes in block k
    C = Counter(z)

    L = 0
    
    # E[k,l] counts how many links exist between nodes in block k and block l
    E = np.zeros((B,B))

    # N[k,l] counts how many links can possibly exist between nodes in block k and l
    N = np.zeros((B,B))
    
    for k in range(B):
        for l in range(B):
            # calculate possible links
            if k == l:
                N[k,l] = sp.special.binom(C[k], 2)
            else:
                N[k,l]= C[k] * C[l]
            for e in n.edges:
                if (z[n.nodes.index[e.v.uid]] == k and z[n.nodes.index[e.w.uid]] == l) or (z[n.nodes.index[e.v.uid]] == l and z[n.nodes.index[e.w.uid]] == k):
                    E[k,l] += 1
    #print(C)
    #print(E)
    #print(N)
    M = E / N
    for k in range(B):
        for l in range(k+1):
            L+= E[k,l] * log(M[k,l]) + (N[k,l] - E[k,l]) * log(1-M[k,l])
    return L, M


def simulated_annealing(network: pp.Network, B, cooling_slowness = 100, changes_per_iter=1, iterations=1000):
    likelihoods = []
    n = network.number_of_nodes()
    z = np.zeros(n, dtype=int)
    for i in range(n):
        z[i] = int(np.random.randint(B))
    l, m = SBM_likelihood(z, network)

    # we output the cooling temperatures to inspect the cooling schedule
    temperatures = []

    for i in tqdm(range(iterations)):

        t = (1+cooling_slowness)/(i+1+cooling_slowness)

        # randomly change community of random node
        z_new = z.copy()
        z_new[np.random.randint(n)] = int(np.random.randint(B))

        for i in range(changes_per_iter):
            i = np.random.choice(network.number_of_nodes())
            z_new[i] = np.random.choice(B)
        l_new, m_new = SBM_likelihood(z_new, network)
        if l_new >= l or np.random.random() <= np.exp(-(l-l_new)/t):
            z = z_new
            l = l_new

        likelihoods.append(np.exp(l))
        temperatures.append(t)
    return z

In [7]:
z_detected = simulated_annealing(network, 2, iterations=3000)
z_detected

  0%|          | 0/3000 [00:00<?, ?it/s]/tmp/ipykernel_44406/1232824955.py:35: RuntimeWarning: invalid value encountered in true_divide
  M = E / N
100%|██████████| 3000/3000 [00:02<00:00, 1377.22it/s]


array([0, 0, 0, 1, 1, 1, 1])

As you see the detected network blocks maximize the likelihood function for two clusters.

### (b) Consider so-called *micro-canonical stochastic block model* i.e. a generalization of the G(n, m) model in the spirit of the stohastic block matrix, where the block matrix entries $M_{kl}$ give the number of edges randomly generated between nodes in block k and block l rather than link probabilitis. Give an expression for the microstate probability of this model.

Let`s consider the arbitrary pair of clusters l and k. The overall number of pairs of stubs is $N_{lk} = \binom{N_{nodes}}{2}$ if l and k is mean the same cluster and $N_{lk} = N_{lnodes} * N_{knodes}$. At the same time the $E_{lk}$ is the number of links inside cluster l is l and k are the same cluster, and number of links between clusters if l != k. The number of variants of such clusters is $\binom{N_{lk}}{E_{lk}}$. So the all clusters exists in the graph in the same time so the number of all variabts of the network is $\prod_{0 <= k <= l <= B - 1} \binom{N_{lk}{}}{E_{lk}}$.

## 2. Simulated annealing

### (a) Use the toy example network with six nodes from the lecture and calculate the maximum likelihood of a stochastic block model with block assignment vector $\hat{z} = (0, 1, 2, 3, 4, 5)$. Compute the maximum likelihood estimation of the stochastic block matrix entries and interpret the resulting matrix $\hat{M}$.

In [8]:
toy_network = pp.Network(directed=False)
toy_network.add_edge("0", "1")
toy_network.add_edge("1", "2")
toy_network.add_edge("0", "2")
toy_network.add_edge("1", "3")
toy_network.add_edge("3", "4")
toy_network.add_edge("3", "5")
toy_network.add_edge("4", "5")
toy_network.plot(edge_color="grey")

In [9]:
z_for_toy_example = (0, 1, 2, 3, 4, 5)

color_graph_communities(toy_network, z_for_toy_example)
toy_network.plot(edge_color="grey")

In [10]:
M_hat = estimate_block_matrix(toy_network, z_for_toy_example)
M_hat

/tmp/ipykernel_44406/3382429916.py:33: RuntimeWarning: invalid value encountered in double_scalars
  M[k,l] = E[k,l] / N[k,l]


array([[nan,  1.,  1.,  0.,  0.,  0.],
       [ 1., nan,  1.,  1.,  0.,  0.],
       [ 1.,  1., nan,  0.,  0.,  0.],
       [ 0.,  1.,  0., nan,  1.,  1.],
       [ 0.,  0.,  0.,  1., nan,  1.],
       [ 0.,  0.,  0.,  1.,  1., nan]])

As you see we defined the maximum likelihood estimation of the stochastic block matrix. This matrix looks exactly like the adjacency matrix with not defined diagonal elements.

In [11]:
A = toy_network.adjacency_matrix().todense()
A

matrix([[0., 1., 1., 0., 0., 0.],
        [1., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1., 0.]])

And as you see the matrices are almost equal.

### (b) In the discussion so far, we have fixed the number of blocks B for the inference of the block assignment vector, i.e. we had to specify the number of communities to be detected in advance. Addressing this isse, adapt the simulated annealing algorithm from the practice session, such that the number of blocks B can change during the optimization process. One approach to achieve this is to start with a block assignment vector that assigns all nodes to a single community. In each step you can randomly split or merge communities and accept the change depending on the current temperature and the difference in likelihood. The result will be an optimization algorithm that detects both the number of block B and the optimal block assignment vector. Apply this algorithm to the toy example and report your finding. interpret the result in the context of task 2 (a).

In [27]:
def extended_simulated_annealing(network: pp.Network, cooling_slowness = 100, changes_per_iter=1, iterations=1000):
    likelihoods = []
    n = network.number_of_nodes()
    z = np.zeros(n, dtype=int)
    l, m = SBM_likelihood(z, network)

    def make_changes(current_z):
        # randomly change community of random node
        chosen_nodes_pair = np.random.choice(np.arange(n), 2, replace=False)
        first_rand_node_idx = chosen_nodes_pair[0]
        second_rand_node_idx = chosen_nodes_pair[1]

        # if the nodes are in the same comunity - split it, if not - merge
        if current_z[first_rand_node_idx] == current_z[second_rand_node_idx]:
            current_z[first_rand_node_idx] = max(current_z) + 1
        else:
            current_z[first_rand_node_idx] = current_z[second_rand_node_idx]

        return current_z

    # we output the cooling temperatures to inspect the cooling schedule
    temperatures = []

    for i in tqdm(range(iterations)):

        t = (1+cooling_slowness)/(i+1+cooling_slowness)

        z_new = make_changes(z.copy())

        for i in range(changes_per_iter):
            z_new = make_changes(z_new)

        l_new, m_new = SBM_likelihood(z_new, network)
        if l_new >= l or np.random.random() <= np.exp(-(l-l_new)/t):
            z = z_new
            l = l_new

        likelihoods.append(np.exp(l))
        temperatures.append(t)
    return z

In [28]:
network_z_from_extended_sa = extended_simulated_annealing(toy_network, changes_per_iter=3, iterations=5000)
network_z_from_extended_sa

  0%|          | 0/5000 [00:00<?, ?it/s]/tmp/ipykernel_44406/1232824955.py:35: RuntimeWarning: invalid value encountered in true_divide
  M = E / N
100%|██████████| 5000/5000 [00:12<00:00, 412.89it/s]


array([1, 1, 1, 0, 0, 0])

In [29]:
color_graph_communities(toy_network, network_z_from_extended_sa)
toy_network.plot(edge_color="grey")

So, we found the $\hat{z}$ vector which maximizes the likelihood function with not fixed number of blocks. So the $\hat{z} = (0, 1, 2, 3, 4, 5)$ is not that one which maximizes the likelihood function.

### (c) Investigate k-means clustering, a simple algorithm to detect clusters in Euclidean data. Explain how the detection of the optimal block number B in the stochastic block model is related to the detection of the optimal cluster number k in k-means. What is the minimal value of the loss function for k-means clustering for k = n, i.e. if the number of clusters corresponds to the number of data points. Explain how this is related to your result from 2(a) and 2(b).

The k-means algorithm is the algorithm for clusterization in the euclidian space. The main idea is that we know exactly the number of clusters and that the form of clusters is convex. First we predefined the centers of the clusters before the all iterations. The main loss function is the Mean Squared Error. For each iteration we check that each point is closed to the center of it`s cluster. If not - change the cluster and recalculate the cluster centers. If on one iteration we change nothing than the algorithm finished the work. 

If we use the fixed number of blocks in stohastic block model we can optimize the network only till some threshold, because the natural number of clusters will be higher/lower than predefined and nodes from another clusters will make the results worser. Same happened for the k-means, when we mix the nodes from different clusters and the centers of the clusters are wrong compared to real situation. 

The loss function for the n clusters from n points will be 0.

As you see the notification in 2.a that each node is separated cluster did not allow us to receive good results in 2.a for the graph. Not predefining the number of clusters helped us to maximize the likelihood function.